In [ ]:
import nfl_data_py as nfl
import pandas as pd
import os
import urllib.request
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnnotationBbox
from matplotlib.offsetbox import OffsetImage

from PIL import Image
import numpy as np
from io import BytesIO
import requests
# ML Libraryies
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import hvplot.pandas
import plotly.express as px

# Graphing 
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.pyplot import figure

In [ ]:
# import data
weekly_nfl = nfl.import_weekly_data([2023])
weekly_nfl.dtypes

In [ ]:
### RB stats
# gather RB stats
rb_df_two = weekly_nfl[(weekly_nfl["position"] == "RB")]
new_columns = ['week','player_display_name','recent_team', 'opponent_team','fantasy_points', 'fantasy_points_ppr','completions','attempts','passing_yards','passing_epa','receptions','passing_tds','targets','rushing_tds','rushing_fumbles','rushing_epa','rushing_yards','carries']
rb_df = rb_df_two[new_columns]
rb_df.head()

In [ ]:
## Groupby opponent team and get mean

# Groupby opponent and week offense stats
team_by_week = rb_df.groupby(['week', 'opponent_team'])
team_by_week.head()
team_stats_per_game = team_by_week.sum()
team_stats_per_game.index.set_names(['week', 'team_defense'], inplace=True)
team_stats_per_game = team_stats_per_game.reset_index()
team_stats_per_game.head()

In [ ]:
# get mean of defense

team_rb_mean = team_stats_per_game.groupby('team_defense').mean()
team_rb_mean.columns = [col + '_def' for col in team_rb_mean.columns]
team_rb_mean = team_rb_mean.drop('week_def', axis=1)
team_rb_mean = team_rb_mean.reset_index()
team_rb_mean.head()
team_rb_mean.to_excel('rb_def.xlsx', index=True)

In [ ]:
# get mean of team rb stats
# Groupby opponent and week offense stats
team_rb = rb_df.groupby(['week', 'recent_team'])
team_rb.head()
team_stats_rb= team_rb.sum()
team_stats_rb.index.set_names(['week', 'team_rb'], inplace=True)

team_stats_rb = team_stats_rb.reset_index()
team_stats_rb.head()

In [ ]:
# get mean of rb

team_rb_df = team_stats_rb.groupby('team_rb').mean()
team_rb_df = team_rb_df.drop('week', axis=1)
team_rb_df = team_rb_df.reset_index()
team_rb_df.head()
team_rb_df.to_excel('rb_offense.xlsx', index=True)

In [ ]:
# get team logo
# Get NFL Logos for DF
logos = nfl.import_team_desc()[['team_abbr', 'team_logo_espn']]
logo_paths = []
team_abbr = []
if not os.path.exists("logos"):
    os.makedirs("logos")

for team in range(len(logos)):
    urllib.request.urlretrieve(logos['team_logo_espn'][team], f"logos/{logos['team_abbr'][team]}.tif")
    logo_paths.append(f"logos/{logos['team_abbr'][team]}.tif")
    team_abbr.append(logos['team_abbr'][team])

data = {'team_abbr' : team_abbr, 'logo_path' : logo_paths}
logo_data = pd.DataFrame(data)

team_rb_df = pd.merge(team_rb_df, logo_data, left_on = 'team_rb', right_on = 'team_abbr')
team_rb_mean = pd.merge(team_rb_mean, logo_data, left_on = 'team_defense', right_on = 'team_abbr')

In [ ]:
team_rb_mean.head()

In [ ]:
# create graph for D vs RB

import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

plt.rcParams["figure.figsize"] = [12, 9]
plt.rcParams["figure.autolayout"] = True

def getImage(path):
    return OffsetImage(plt.imread(path, format="tif"), zoom=.08)

y = team_rb_mean['carries_def']
x = team_rb_mean['rushing_yards_def']
paths = team_rb_mean['logo_path']

fig, ax = plt.subplots()

# Define quadrant boundaries
x_mid = (x.max() + x.min()) / 2
y_mid = (y.max() + y.min()) / 2

# Quadrant 1 (Top-Right)
for x0, y0, path in zip(x, y, paths):
    if x0 >= x_mid and y0 >= y_mid:
        ab = AnnotationBbox(getImage(path), (x0, y0), frameon=False)
        ax.add_artist(ab)

# Quadrant 2 (Top-Left)
for x0, y0, path in zip(x, y, paths):
    if x0 < x_mid and y0 >= y_mid:
        ab = AnnotationBbox(getImage(path), (x0, y0), frameon=False)
        ax.add_artist(ab)

# Quadrant 3 (Bottom-Left)
for x0, y0, path in zip(x, y, paths):
    if x0 < x_mid and y0 < y_mid:
        ab = AnnotationBbox(getImage(path), (x0, y0), frameon=False)
        ax.add_artist(ab)

# Quadrant 4 (Bottom-Right)
for x0, y0, path in zip(x, y, paths):
    if x0 >= x_mid and y0 < y_mid:
        ab = AnnotationBbox(getImage(path), (x0, y0), frameon=False)
        ax.add_artist(ab)

        
plt.axvline(x_mid, color='black', linestyle='--', alpha=0.5)
plt.axhline(y_mid, color='black', linestyle='--', alpha=0.5)

plt.title("Defense vs. RB Carries/Yards")
plt.ylabel("Carries vs. D")
plt.xlabel("Rush Yards vs. RB")
plt.grid(False)
plt.xlim(40, 150)
plt.ylim(8, 27)
# Quadrant 1 
plt.text(x_mid + 12, y_mid + 1, 'Bad vs RB', fontsize=15, color='green')

# Quadrant 2 
plt.text(x_mid - 28, y_mid + 4, 'Okay vs RB', fontsize=15, color='blue')

# Quadrant 3 
plt.text(x_mid - 30, y_mid - 10, 'Great vs RB', fontsize=15, color='red')

# Quadrant 4 
#plt.text(x_mid + 30, y_mid - 10, 'Good vs RB', fontsize=15, color='orange')

plt.savefig('D_vs.png')
plt.show()

In [ ]:
# create graph for RB team

import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

plt.rcParams["figure.figsize"] = [12, 9]
plt.rcParams["figure.autolayout"] = True

def getImage(path):
    return OffsetImage(plt.imread(path, format="tif"), zoom=.08)

y = team_rb_df['carries']
x = team_rb_df['rushing_yards']
paths = team_rb_df['logo_path']

fig, ax = plt.subplots()

# Define quadrant boundaries
x_mid = (x.max() + x.min()) / 2
y_mid = (y.max() + y.min()) / 2

# Quadrant 1 (Top-Right)
for x0, y0, path in zip(x, y, paths):
    if x0 >= x_mid and y0 >= y_mid:
        ab = AnnotationBbox(getImage(path), (x0, y0), frameon=False)
        ax.add_artist(ab)

# Quadrant 2 (Top-Left)
for x0, y0, path in zip(x, y, paths):
    if x0 < x_mid and y0 >= y_mid:
        ab = AnnotationBbox(getImage(path), (x0, y0), frameon=False)
        ax.add_artist(ab)

# Quadrant 3 (Bottom-Left)
for x0, y0, path in zip(x, y, paths):
    if x0 < x_mid and y0 < y_mid:
        ab = AnnotationBbox(getImage(path), (x0, y0), frameon=False)
        ax.add_artist(ab)

# Quadrant 4 (Bottom-Right)
for x0, y0, path in zip(x, y, paths):
    if x0 >= x_mid and y0 < y_mid:
        ab = AnnotationBbox(getImage(path), (x0, y0), frameon=False)
        ax.add_artist(ab)

        
plt.axvline(x_mid, color='black', linestyle='--', alpha=0.5)
plt.axhline(y_mid, color='black', linestyle='--', alpha=0.5)

plt.title("RB Carries/Yards per Team")
plt.ylabel("Carries Team")
plt.xlabel("Rush Yards Team")
plt.grid(False)
plt.xlim(40, 200)
plt.ylim(8, 30)
# Quadrant 1 
plt.text(x_mid + 12, y_mid + 1, 'High Carrie, High Yards', fontsize=15, color='green')

# Quadrant 2 
plt.text(x_mid - 38, y_mid + 4, 'High Carries, Low Yards', fontsize=15, color='blue')

# Quadrant 3 
plt.text(x_mid - 30, y_mid - 10, 'Low RB Volume', fontsize=15, color='red')

# Quadrant 4 
#plt.text(x_mid + 30, y_mid - 10, 'Good vs RB', fontsize=15, color='orange')

plt.savefig('RB_vs_D.png')
plt.show()

In [ ]:
# create graph for D vs RB

import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

plt.rcParams["figure.figsize"] = [12, 9]
plt.rcParams["figure.autolayout"] = True

def getImage(path):
    return OffsetImage(plt.imread(path, format="tif"), zoom=.08)

y = team_rb_df['rushing_epa']
x = team_rb_df['fantasy_points_ppr']
paths = team_rb_df['logo_path']

fig, ax = plt.subplots()

# Define quadrant boundaries
x_mid = (x.max() + x.min()) / 2
y_mid = (y.max() + y.min()) / 2

# Quadrant 1 (Top-Right)
for x0, y0, path in zip(x, y, paths):
    if x0 >= x_mid and y0 >= y_mid:
        ab = AnnotationBbox(getImage(path), (x0, y0), frameon=False)
        ax.add_artist(ab)

# Quadrant 2 (Top-Left)
for x0, y0, path in zip(x, y, paths):
    if x0 < x_mid and y0 >= y_mid:
        ab = AnnotationBbox(getImage(path), (x0, y0), frameon=False)
        ax.add_artist(ab)

# Quadrant 3 (Bottom-Left)
for x0, y0, path in zip(x, y, paths):
    if x0 < x_mid and y0 < y_mid:
        ab = AnnotationBbox(getImage(path), (x0, y0), frameon=False)
        ax.add_artist(ab)

# Quadrant 4 (Bottom-Right)
for x0, y0, path in zip(x, y, paths):
    if x0 >= x_mid and y0 < y_mid:
        ab = AnnotationBbox(getImage(path), (x0, y0), frameon=False)
        ax.add_artist(ab)

        
plt.axvline(x_mid, color='black', linestyle='--', alpha=0.5)
plt.axhline(y_mid, color='black', linestyle='--', alpha=0.5)

plt.title("Team RB Advanced")
plt.ylabel("EPA vs. D")
plt.xlabel("PPR vs. RB")
plt.grid(False)
plt.xlim(10, 40)
plt.ylim(-10, 10)
# Quadrant 1 
#plt.text(x_mid + 3, y_mid + 10, 'Bad vs TE', fontsize=15, color='green')

# Quadrant 2 
#plt.text(x_mid - 3, y_mid + 10, 'Okay vs TE', fontsize=15, color='blue')

# Quadrant 3 
#plt.text(x_mid - 3, y_mid - 10, 'Great vs TE', fontsize=15, color='red')

# Quadrant 4 
#plt.text(x_mid + 3, y_mid - 10, 'Good vs TE', fontsize=15, color='orange')

#plt.savefig('TE_vs_D.png')
plt.show()